# Book 5: Building the AI API
## בספר זה מדבר על בניית הclass של בapi
זה הוא ספר האחרון אשר מחבר את כל התהליכים בספרים הקודמים לתהליך אחד אשר מרכיב את ה-ai

## import

In [2]:
import os #צויין קודם
import PrallelCode #ספר 2
import MachineLearning #ספר 3
import random # ספרייה הרחבה לפעלות עקריות
import pickle #צויין קודם
import enum #צויין קודם
import numpy as np #צויין קודם
import tensorflow as tf #צויין קודם
import GetNamesByClass #ספר 4
import csv # ספריית הרחבה לפעולות על csv
import config #צויין קודם
import HebrewNlp #ספר 1
import CleanText # ספר 1


class Location(enum.IntEnum): 
    Lable = 0 #סוג הדוח
    Root = 1 #מיקום הקובץ
    wordtoken =2 #המילים בקובץ
    bagofword = 3 #מבני הנתונים
    clientname =4 #שם הלקוח


## model traing class
זה הclass של הapi
פה בעצם יצירת המודל מתקיימית

In [3]:
class ModelTrainig:
    def __int__(self,): #אובייקטים של ה- class
        self.bagofwords = {} #הtamplte של כל המילים בעולם שלנו
        self.trainiset= [] #מידע ללאימון המוכנה
        self.category_index # אינדקס המידע לזיהוי נתונים
        self.detection_model # מוכנה של ספר 4
        self.allfirstname # כל השמות הפרטים העיברים
        self.alllastname# כל השמות המשפחה העיברים

## class function
### פנקציות לטעינת המוכנה:

הפנוקציה הזאת טעונת את ה-template

ומאגר המילים לתוך האובייקטים של הקלאס

In [5]:
    def Loadbegofworld(self):
        if (not os.path.exists(os.path.join(config.WORKSPACE,config.SUFFLETRAINFILE))): #בדיקה אם הקבצים שראינו בספר 2 קיימים
            (self.bagofwords, self.trainiset) = PrallelCode.begofword() #אם לא קיימים אז להפעיל את התהליך שבספר 2 על כל הdata set וליצור קבצים מתאימים
            random.shuffle(self.trainiset) # עירוב הדאטה סט בשביל לעלות את האמינות המכונה
            file = open(os.path.join(config.WORKSPACE, config.SUFFLETRAINFILE), 'wb') #ירצרת קובץ חדש של הדאטה מעורבב
            pickle.dump(self.trainiset, file)
            file.close()
        else:
            file = open(os.path.join(config.WORKSPACE, config.BAGOFWORDSFILE), 'rb') # הכנסת הנתונים המתאים מהקבצים הנצרו בספר 2
            self.bagofwords = pickle.load(file)
            file.close()
            file = open(os.path.join(config.WORKSPACE, config.SUFFLETRAINFILE), 'rb') # הכנסת הנתונים המתאים מהקבצים הנצרו בספר 2
            self.trainiset = pickle.load(file)
            file.close()
            

## בניית המודל של ספר 3
הפונקציה מכינה מחלק את הדאטה לשני המערכים במתאים לספר 3. ומכינה את המוכנה

In [ ]:
 def buildthemodel(self):
        labels=[]  #מערך תוויות
        files = [] # מערך האינפוט
        for item in self.trainiset: #יצירת מערך לשליחה לספר 3
            npitem = np.array(item[int(Location.bagofword)])
            files.append(item[int(Location.bagofword)])
            if item[int(Location.Lable)] == config.REPORTTYPE2: # בדיקת התווית והכנסה לתוך מערך התווים
                labels.append(1)
            else:
                labels.append(0)
        MachineLearning.buildmodel(labels,files)    #פונקציה הכנת במודל מספר 3     
    

  

## בניית המודל של ספר  3 -4
הפונקציה הזאת היא טוענת את המודל של ספר 4 וספר 3 

In [8]:
def LoadModel(self): 
        try:
            self.model = tf.keras.models.load_model(config.MODELNANE) #נסיון טעינה של ספר 3 אם קיים
        except Exception as e:
            self.model = self.buildthemodel() #יצירת מודל של ספר 3 עם לא קיים
        self.category_index, self.detection_model = GetNamesByClass.Load_Pic_Model() # טעינה ספר 4

## טעינת הקבצים של 

In [9]:
def LoadNames(self):
        allname = [] #מערך שמות פרטים
        with open(config.FIRSTNAME, encoding=config.ENCODING, newline='') as csvfile:# טעינה מקובץ csv
            reader = csv.reader(csvfile, delimiter='\t')
            for row in reader:
                data = ', '.join(row)
                allname.append(data)
        self.allfirstname = allname
        allname = [] # מערך שמות המשפחה
        with open(config.LASTNAME, encoding=config.ENCODING, newline='') as csvfile: # טעינה מקובץ csv
            reader = csv.reader(csvfile, delimiter='\t')
            for row in reader:
                data = ', '.join(row)
                allname.append(data)
        self.alllastname = allname

## טעינת הפרויקט
הפומקתחה הזאת טוענת את כל המכנות ואת כל הקבצים לפרויקט

In [10]:
  def LoadTheApi(self):
        self.Loadbegofworld()
        self.LoadModel()
        self.LoadNames()
        try: #בדיקת אם הapi החיצונים תקנים
            HebrewNlp.HebrewTokenizer(config.ERRORLABEL) 
            CleanText.Translate2Word("english")
        except Exception as e:
             raise e

## מיצאת השם הקובץ
הפונקציה הזאת מקבלת כתובת לקובץ שם קובץ 
הפקציה מוציאה את השם מהקובץ לפי ספר 4

In [11]:
 def NameWithCat(self, file, name, root):
        append_lst = file.copy()
        append_lst.append(
            GetNamesByClass.Classtfy( #קריאה לספר 4
                self.category_index,
                self.detection_model,
                root, name,
                self.allfirstname,
                self.alllastname))
        return append_lst

##  חזוי מסמך חדש
הפונקציה מקבלת תקייה למיון המסמכים ופונקציה אומרת לען כל מסמך צריך להגיע

In [13]:
 def predictAndClasterfy(self,when):
        data = PrallelCode.bagofwordonlytheard(when) #הכנת האינפוט למערכת ספר 1-2
        data_after_subject = MachineLearning.predict(self.model, data) #חיזוי ספר 3
        allthesetup = []
        for file in data_after_subject: # יצירת המידע לשליחה לסרבר-לקוח
            if file[int(Location.Lable)] in config.REPORTTYPE1: #הכנסה תוותים מתאימה לסוג מסמך והוצאת המידע לפי סוג מסמך
                root =file[int(Location.Root)]
                allname = root.split('\\')
                name = allname[(len(allname))-1]
                append_lst = self.NameWithCat(file, name, root) #שליחה לפונקציה הוצאת השם
                allthesetup.append(append_lst) # הופסת השם לרישמה
            else:
                append_lst = file.copy()
                append_lst.append(HebrewNlp.GetName(file[int(Location.wordtoken)],self.allfirstname,self.alllastname))
                allthesetup.append(append_lst) #הוצאת שם לפי ספר 1
        return allthesetup

    

    

## תוצאה של כל ה- AI יחד

In [15]:
    import warnings
    import LoudModel
    warnings.filterwarnings("ignore")
    model = LoudModel.ModelTrainig()
    model.LoadTheApi()
    allfile = model.predictAndClasterfy(os.path.join("File\here"))
    print(allfile)

File\2015.jpeg
File\2018.jpeg
[['לבדיקה', 'File\\here\\fdgfdgfdgfd.pdf', ['כה', 'ביגי', 'ושותי', 'אש', 'שמונה', 'ושוהפחסט', 'בה', 'לגל', 'רוא', 'חשבו', 'אפס', 'לכבוד', 'מיכל', 'גוב', 'תאח', 'אפס', 'מספר', 'עמ', 'אפס', 'תפ', 'עשרי', 'ושלושה', 'מיליו', 'שש', 'מאות', 'ועשרי', 'וארבעת', 'אלפי', 'ארבע', 'מאות', 'ושבעי', 'ואחת', 'חשבונית', 'מס', 'שתי', 'עשרה', 'אל', 'שש', 'מאות', 'וחמישי', 'ושתיי', 'מקור', 'נפרע', 'עי', 'אפס', 'סהכ', 'חיב', 'מע', 'פטור', 'ממסי', 'במקור', 'לפי', 'אשור', 'פקיד', 'שומה', 'עידו', 'תנאי', 'תשלו', 'מחמ', 'מעמ', 'בואט', 'מו', 'ספ', 'העברה', 'בגקאית', 'חשמנות', 'מס', 'שתי', 'עשרה', 'אל', 'שש', 'מאות', 'וחמישי', 'ושתיי', 'מפק', 'המסמ', 'עח', 'העחת', 'סהכ', 'חתימה', 'ררר', 'אפס', 'שתיי', 'שבעה', 'דהה', 'היה', 'מסמ', 'זה', 'הופק', 'עי', 'תוכנת', 'סופ', 'כו', 'עמזד', 'אחד', 'קהילת', 'סלונקי', 'תשע', 'טלפ', 'שלושה', 'תל', 'ב', 'פקס', 'חפ'], array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0